In [20]:
import ee
import time
import json
import gdal, ogr, osr

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# Cargo el shapefile. Un shapefile es el formato de almacenamiento
# desarrollado por ESRI para vector data. Es en realidad una coleccion
# de archivos
shapefiles_path = '/content/drive/MyDrive/CEIA/Proyecto/proyecto/shapefile'
shapefile = ogr.Open(shapefiles_path)

In [10]:
# OGR model se compone de Data Sources, Layers y Features. 
# Features tiene attributes y Geometry
# Vector data model https://gdal.org/user/vector_data_model.html

# miro layers
numLayers = shapefile.GetLayerCount()
print(f'Shapefiles contains {numLayers} layers')
layer = shapefile.GetLayer(0)

# extraigo informacion geoespacial del layer
spatialRef = layer.GetSpatialRef().ExportToProj4()
numFeatures = layer.GetFeatureCount()
print(f'Layer has spatial reference {spatialRef} layers')
print(f'Layer has {numFeatures} reference {spatialRef} layers')

# miro metadata (features)
feature = layer.GetFeature(0)
featureItems = feature.items()
for key, value in featureItems.items():
    print(f'{key} {value}')

Shapefiles contains 1 layers
Layer has spatial reference +proj=longlat +datum=WGS84 +no_defs  layers
Layer has 1 reference +proj=longlat +datum=WGS84 +no_defs  layers
objectid 406
departa GENERAL LOPEZ
cabecer MELINCUE
provincia SANTA FE
fuente CATASTRO SANTA FE
fec_actual JUN 2007
fuente1 CATASTROS PROVINCIALES
cod_depto 82042
created_us None
created_da None
last_edite None
last_edi_1 None
st_area_sh 1.1148561411
st_length_ 4.46046207017


In [11]:
 # miro la geometria
def analyzeGeometry(geometry, indent=0):
    s = []
    s.append(" " * indent)
    s.append(geometry.GetGeometryName())
    if geometry.GetPointCount() > 0:
        s.append(" with %d data points" % geometry.GetPointCount())
    if geometry.GetGeometryCount() > 0:
        s.append(" containing:")
    print("".join(s))
    for i in range(geometry.GetGeometryCount()):
        analyzeGeometry(geometry.GetGeometryRef(i), indent+1)
        
geom = feature.GetGeometryRef()
analyzeGeometry(geom, 3)


   POLYGON containing:
    LINEARRING with 155 data points


In [ ]:
ee.Authenticate()
ee.Initialize()

gj = feature.GetGeometryRef().ExportToJson()
gjObject = json.loads(gj)

#  en EE un Feature esta definido por un GeoJSON que contiene una geometria 
# y propiedades almacenadas en un diccionario
# GeoJSON (RFC7946) https://geojson.org/
geometry = ee.Geometry(gjObject)

# selecciono el catalogo de copernicus sentinel2
image_collection = ee.ImageCollection('COPERNICUS/S2')

# defino filtro temporal
# Cada camapaña va desde mayo a abril de cada año. 
#Campaña 18/19
camp_1819 = ['2018-05-01', '2019-05-01']
#Campaña 19/20
camp_1920 = ['2019-05-01','2020-05-01']

# filtro el catalogo por geometria, tiempo nubosidad y bandas
camp1819_imagenes = image_collection \
    .filterBounds(geometry) \
    .filterDate(camp_1819[0], camp_1819[1]) \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than', 10)\
    .select(['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9', 'B11','B12'])

camp1920_imagenes = image_collection \
    .filterBounds(geometry) \
    .filterDate(camp_1920[0], camp_1920[1]) \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than', 10)\
    .select(['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9', 'B11','B12'])


# batch permite usar el sistema de ee para ejecutar tareas
# de este modo podemos usar la funcionalidad de la case Export
# para almacenar las imagenes en batches
def export(collection):
    params = {
        'description': 'myExportImageTask',
        'folder':'imagenes',
        'scale': 30,
        'fileFormat': 'GeoTIFF',
        'region': geometry.getInfo()["coordinates"]
    }
    cList = collection.toList(collection.size())
    i = 0
    while i>=0:
        print(f'guardo imagen{i}')
        try:
            image = ee.Image(cList.get(i))
            task = ee.batch.Export.image.toDrive(image, **params)
            task.start()
            while task.active():
                time.sleep(30)
                print(task.status())
            i = i + 1
        except:
            print('LISTO')
    
    
export(camp1819_imagenes)
export(camp1920_imagenes)


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=G6USSp-KM9EKa6nOo3mWz-QZdymoGjIZYjqaZbxOjz0&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjj1sfbznHqM16iGTq8UEP8vc_nJmNuRUGyTpbLKY0El31-P7_heVk

Successfully saved authorization token.
guardo imagen0
{'state': 'RUNNING', 'description': 'myExportImageTask', 'creation_timestamp_ms': 1632693073468, 'update_timestamp_ms': 1632693097671, 'start_timestamp_ms